# LSTM Neural Network Modeling for Chili Price Forecasting

**Algorithm:** LSTM (Long Short-Term Memory)

**Architecture:**
- Multivariate approach: ONE model predicts all 5 markets simultaneously
- Two models: LSTM without holidays, LSTM with holidays
- Input shape: (LOOK_BACK, num_markets) or (LOOK_BACK, num_markets + 1)
- Output: Dense(num_markets) predicting all market prices

**Prerequisites:** Run `01_data_cleaning_and_eda.ipynb` first


In [6]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib
import os

# Ensure directories exist
os.makedirs('../models/lstm', exist_ok=True)
os.makedirs('../results/metrics', exist_ok=True)

print('✓ Libraries imported successfully')
print(f'✓ TensorFlow version: {tf.__version__}')

✓ Libraries imported successfully
✓ TensorFlow version: 2.20.0


In [7]:
# Load preprocessed data
df_with_holidays = pd.read_csv('../data/processed/data_with_holidays.csv', index_col=0, parse_dates=True)

print(f"Data loaded: {df_with_holidays.shape}")
print(f"Date range: {df_with_holidays.index.min()} to {df_with_holidays.index.max()}")
df_with_holidays.head()

Data loaded: (471, 6)
Date range: 2024-01-01 00:00:00 to 2025-10-24 00:00:00


,Pasar Aksara,Pasar Brayan,Pasar Petisah,Pasar Sukaramai,Pusat Pasar,is_holiday
Date,,,,,,
2024-01-01,30000.0,26500.0,30000.0,30000.0,27500.0,1
2024-01-02,35000.0,32500.0,35000.0,38000.0,30000.0,0
2024-01-03,35000.0,32500.0,35000.0,38000.0,30000.0,0
2024-01-04,30000.0,30000.0,30000.0,38000.0,30000.0,0
2024-01-05,30000.0,30000.0,30000.0,30000.0,24500.0,0


In [8]:
# Define parameters
market_columns = ['Pasar Sukaramai', 'Pasar Aksara', 'Pasar Petisah', 'Pusat Pasar', 'Pasar Brayan']
TEST_SIZE = 0.2
LOOK_BACK = 30  # Use 30 days of history
SPLIT_INDEX = int(len(df_with_holidays) * (1 - TEST_SIZE))
EPOCHS = 50
BATCH_SIZE = 16

print(f"Markets: {len(market_columns)}")
print(f"Look-back window: {LOOK_BACK} days")
print(f"Epochs: {EPOCHS}")
print(f"Batch size: {BATCH_SIZE}")

# Split data chronologically
train_data = df_with_holidays.iloc[:SPLIT_INDEX]
test_data = df_with_holidays.iloc[SPLIT_INDEX:]

print(f"\nTraining data: {train_data.shape[0]} days ({train_data.index[0]} to {train_data.index[-1]})")
print(f"Testing data: {test_data.shape[0]} days ({test_data.index[0]} to {test_data.index[-1]})")

Markets: 5
Look-back window: 30 days
Epochs: 50
Batch size: 16

Training data: 376 days (2024-01-01 00:00:00 to 2025-06-13 00:00:00)
Testing data: 95 days (2025-06-16 00:00:00 to 2025-10-24 00:00:00)


In [9]:
# Helper function for MAPE calculation
def calculate_mape(actual, predicted):
    """Calculate Mean Absolute Percentage Error"""
    mask = actual != 0
    if mask.sum() == 0:
        return np.nan
    mape = np.mean(np.abs((actual[mask] - predicted[mask]) / actual[mask])) * 100
    return min(mape, 999.99)

# Load the scalers created in notebook 01
print("Loading scalers from data/scalers/...")
scaler_markets = joblib.load('../data/scalers/scaler_markets.joblib')
scaler_with_features = joblib.load('../data/scalers/scaler_with_features.joblib')
print("✓ Scalers loaded successfully")

# Scale the data using the loaded scalers
print("\nScaling data with pre-fitted scalers...")

# Markets only (for LSTM without holidays)
train_markets_scaled = scaler_markets.transform(train_data[market_columns])
test_markets_scaled = scaler_markets.transform(test_data[market_columns])

print("Markets-only scaling:")
print(f"  Train shape: {train_markets_scaled.shape}")
print(f"  Test shape: {test_markets_scaled.shape}")

# Markets + holiday feature (for LSTM with holidays)
feature_columns = market_columns + ['is_holiday']
train_features_scaled = scaler_with_features.transform(train_data[feature_columns])
test_features_scaled = scaler_with_features.transform(test_data[feature_columns])

print("\nMarkets + holiday scaling:")
print(f"  Train shape: {train_features_scaled.shape}")
print(f"  Test shape: {test_features_scaled.shape}")

print("\n✓ Data scaled successfully using existing scalers!")

Loading scalers from data/scalers/...
✓ Scalers loaded successfully

Scaling data with pre-fitted scalers...
Markets-only scaling:
  Train shape: (376, 5)
  Test shape: (95, 5)

Markets + holiday scaling:
  Train shape: (376, 6)
  Test shape: (95, 6)

✓ Data scaled successfully using existing scalers!


In [10]:
print(f"\n{'='*70}")
print(f"Training LSTM Models - Multivariate Approach (ONE model for ALL markets)")
print(f"{'='*70}\n")

# ===========================
# Model 1: LSTM without holidays (5 features: 5 markets)
# ===========================
print("="*50)
print("Model 1: LSTM without holidays (Markets only)")
print("="*50)

# Prepare data - only market columns (5 features)
data_no_holiday = train_markets_scaled
test_data_nh = test_markets_scaled

n_features_nh = data_no_holiday.shape[1]  # 5 markets
print(f"Training data shape: {data_no_holiday.shape}")
print(f"Test data shape: {test_data_nh.shape}")
print(f"Number of features (markets): {n_features_nh}")

# Create time series generators
train_generator_nh = TimeseriesGenerator(
    data_no_holiday, 
    data_no_holiday,
    length=LOOK_BACK,
    batch_size=BATCH_SIZE
)

test_generator_nh = TimeseriesGenerator(
    test_data_nh,
    test_data_nh,
    length=LOOK_BACK,
    batch_size=BATCH_SIZE
)

print(f"Train generator samples: {len(train_generator_nh)}")
print(f"Test generator samples: {len(test_generator_nh)}")

# Build LSTM model (input: 5 markets, output: 5 markets)
lstm_model = Sequential([
    LSTM(64, activation='relu', input_shape=(LOOK_BACK, n_features_nh), return_sequences=True),
    Dropout(0.2),
    LSTM(32, activation='relu'),
    Dropout(0.2),
    Dense(n_features_nh)  # Predict all 5 markets simultaneously
])

lstm_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
print("\nModel architecture:")
lstm_model.summary()

# Train the model
print("\nTraining LSTM model...")
history = lstm_model.fit(
    train_generator_nh,
    epochs=EPOCHS,
    verbose=1,
    validation_data=test_generator_nh
)

# Make predictions on test set
print("\nMaking predictions on test set...")
lstm_predictions = []
for i in range(len(test_generator_nh)):
    X, _ = test_generator_nh[i]
    pred = lstm_model.predict(X, verbose=0)
    lstm_predictions.extend(pred)

lstm_predictions = np.array(lstm_predictions)
print(f"Predictions shape: {lstm_predictions.shape}")

# Inverse transform predictions and actual values
lstm_pred = scaler_markets.inverse_transform(lstm_predictions)
actual_test_nh = test_data_nh[LOOK_BACK:LOOK_BACK + len(lstm_pred)]
y_test = scaler_markets.inverse_transform(actual_test_nh)

# Calculate metrics for each market
print("\n" + "="*50)
print("LSTM (no holidays) - Metrics by Market:")
print("="*50)
lstm_rmse_list = []
lstm_mae_list = []
lstm_mape_list = []

for idx, market in enumerate(market_columns):
    rmse = np.sqrt(mean_squared_error(y_test[:, idx], lstm_pred[:, idx]))
    mae = mean_absolute_error(y_test[:, idx], lstm_pred[:, idx])
    mape = calculate_mape(y_test[:, idx], lstm_pred[:, idx])
    
    lstm_rmse_list.append(rmse)
    lstm_mae_list.append(mae)
    lstm_mape_list.append(mape)
    
    print(f"{market:25s}: RMSE={rmse:8.2f}, MAE={mae:8.2f}, MAPE={mape:6.2f}%")

avg_lstm = np.mean(lstm_rmse_list)
print(f"\nAverage RMSE: {avg_lstm:.2f}")

# Save the model
lstm_model.save('../models/lstm/lstm_model_all_markets.h5')
print("Model saved to: models/lstm/lstm_model_all_markets.h5\n")


# ===========================
# Model 2: LSTM with holidays (6 features: 5 markets + holiday)
# ===========================
print("\n" + "="*50)
print("Model 2: LSTM with holidays (Markets + Holiday feature)")
print("="*50)

# Prepare data - markets + holiday indicator (6 features)
data_with_holiday = train_features_scaled
test_data_wh = test_features_scaled

n_features_wh = data_with_holiday.shape[1]  # 6 features
print(f"Training data shape: {data_with_holiday.shape}")
print(f"Test data shape: {test_data_wh.shape}")
print(f"Number of features (markets + holiday): {n_features_wh}")

# Create time series generators
train_generator_wh = TimeseriesGenerator(
    data_with_holiday,
    data_with_holiday,
    length=LOOK_BACK,
    batch_size=BATCH_SIZE
)

test_generator_wh = TimeseriesGenerator(
    test_data_wh,
    test_data_wh,
    length=LOOK_BACK,
    batch_size=BATCH_SIZE
)

print(f"Train generator samples: {len(train_generator_wh)}")
print(f"Test generator samples: {len(test_generator_wh)}")

# Build LSTM model with holiday (input: 6 features, output: 6 features)
lstm_holiday_model = Sequential([
    LSTM(64, activation='relu', input_shape=(LOOK_BACK, n_features_wh), return_sequences=True),
    Dropout(0.2),
    LSTM(32, activation='relu'),
    Dropout(0.2),
    Dense(n_features_wh)  # Predict all 6 features
])

lstm_holiday_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
print("\nModel architecture:")
lstm_holiday_model.summary()

# Train the model
print("\nTraining LSTM model with holidays...")
history_h = lstm_holiday_model.fit(
    train_generator_wh,
    epochs=EPOCHS,
    verbose=1,
    validation_data=test_generator_wh
)

# Make predictions on test set
print("\nMaking predictions on test set...")
lstm_holiday_predictions = []
for i in range(len(test_generator_wh)):
    X, _ = test_generator_wh[i]
    pred = lstm_holiday_model.predict(X, verbose=0)
    lstm_holiday_predictions.extend(pred)

lstm_holiday_predictions = np.array(lstm_holiday_predictions)
print(f"Predictions shape: {lstm_holiday_predictions.shape}")

# Inverse transform - only take the first 5 columns (markets)
lstm_holiday_pred_all = scaler_with_features.inverse_transform(lstm_holiday_predictions)
lstm_holiday_pred = lstm_holiday_pred_all[:, :5]  # Extract only market predictions

actual_test_wh = test_data_wh[LOOK_BACK:LOOK_BACK + len(lstm_holiday_pred)]
y_test_h_all = scaler_with_features.inverse_transform(actual_test_wh)
y_test_h = y_test_h_all[:, :5]  # Extract only actual market values

# Calculate metrics for each market
print("\n" + "="*50)
print("LSTM (with holidays) - Metrics by Market:")
print("="*50)
lstm_h_rmse_list = []
lstm_h_mae_list = []
lstm_h_mape_list = []

for idx, market in enumerate(market_columns):
    rmse = np.sqrt(mean_squared_error(y_test_h[:, idx], lstm_holiday_pred[:, idx]))
    mae = mean_absolute_error(y_test_h[:, idx], lstm_holiday_pred[:, idx])
    mape = calculate_mape(y_test_h[:, idx], lstm_holiday_pred[:, idx])
    
    lstm_h_rmse_list.append(rmse)
    lstm_h_mae_list.append(mae)
    lstm_h_mape_list.append(mape)
    
    print(f"{market:25s}: RMSE={rmse:8.2f}, MAE={mae:8.2f}, MAPE={mape:6.2f}%")

avg_lstm_h = np.mean(lstm_h_rmse_list)
print(f"\nAverage RMSE: {avg_lstm_h:.2f}")

# Save the model
lstm_holiday_model.save('../models/lstm/lstm_holiday_model_all_markets.h5')
print("Model saved to: models/lstm/lstm_holiday_model_all_markets.h5\n")

# Store results for later comparison
lstm_results = {
    'predictions_no_holiday': lstm_pred,
    'predictions_with_holiday': lstm_holiday_pred,
    'actual': y_test,  # Use y_test since both have same actual values
    'test_dates': test_data.index[LOOK_BACK:LOOK_BACK + len(lstm_pred)],
    'rmse_no_holiday': lstm_rmse_list,
    'mae_no_holiday': lstm_mae_list,
    'mape_no_holiday': lstm_mape_list,
    'rmse_with_holiday': lstm_h_rmse_list,
    'mae_with_holiday': lstm_h_mae_list,
    'mape_with_holiday': lstm_h_mape_list,
    'avg_rmse_no_holiday': avg_lstm,
    'avg_rmse_with_holiday': avg_lstm_h
}

print("\n" + "="*70)
print("LSTM Training Complete!")
print(f"Total models trained: 2 (one without holidays, one with holidays)")
print("="*70)


Training LSTM Models - Multivariate Approach (ONE model for ALL markets)

Model 1: LSTM without holidays (Markets only)
Training data shape: (376, 5)
Test data shape: (95, 5)
Number of features (markets): 5
Train generator samples: 22
Test generator samples: 5



Model architecture:


2025-11-06 01:15:03.032690: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/usr/local/python/3.13.9/lib/python3.13/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 30, 64)         │        17,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 30, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

 Total params: 30,790 (120.27 KB)

 Trainable params: 30,790 (120.27 KB)

 Non-trainable params: 0 (0.00 B)


Training LSTM model with holidays...
Epoch 1/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 0.1101 - mae: 0.2411 - val_loss: 0.1630 - val_mae: 0.3195
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 0.1101 - mae: 0.2411 - val_loss: 0.1630 - val_mae: 0.3195
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0571 - mae: 0.1685 - val_loss: 0.0816 - val_mae: 0.2314
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0571 - mae: 0.1685 - val_loss: 0.0816 - val_mae: 0.2314
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0509 - mae: 0.1582 - val_loss: 0.0700 - val_mae: 0.2144
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0509 - mae: 0.1582 - val_loss: 0.0700 - val_mae: 0.2144
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0478 - mae: 0.1527 - val_loss: 0.0543 - val_mae: 0.1845
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0478 - mae: 0.1527 - val_loss: 0.0543 - val_mae: 0.1845
Epoch 5/50
22/22 ━━━━━━━━━

Predictions shape: (65, 6)

LSTM (with holidays) - Metrics by Market:
Pasar Sukaramai          : RMSE=16995.30, MAE=13799.58, MAPE= 20.70%
Pasar Aksara             : RMSE=13057.60, MAE=10539.59, MAPE= 16.47%
Pasar Petisah            : RMSE=13765.96, MAE=11157.20, MAPE= 17.15%
Pusat Pasar              : RMSE=15210.98, MAE=12388.01, MAPE= 18.90%
Pasar Brayan             : RMSE=13459.66, MAE=11216.79, MAPE= 16.87%

Average RMSE: 14497.90
Model saved to: models/lstm/lstm_holiday_model_all_markets.h5


LSTM Training Complete!
Total models trained: 2 (one without holidays, one with holidays)


In [11]:
# Calculate average MAPE values
avg_mape_no_holiday = np.mean(lstm_results['mape_no_holiday'])
avg_mape_with_holiday = np.mean(lstm_results['mape_with_holiday'])

print("\n" + "="*70)
print("LSTM FINAL SUMMARY:")
print("="*70)
print(f"LSTM (no holidays):")
print(f"  Average RMSE: {lstm_results['avg_rmse_no_holiday']:,.2f}")
print(f"  Average MAPE: {avg_mape_no_holiday:.2f}%")
print(f"\nLSTM (with holidays):")
print(f"  Average RMSE: {lstm_results['avg_rmse_with_holiday']:,.2f}")
print(f"  Average MAPE: {avg_mape_with_holiday:.2f}%")

improvement = ((lstm_results['avg_rmse_no_holiday'] - lstm_results['avg_rmse_with_holiday']) / lstm_results['avg_rmse_no_holiday']) * 100
print(f"\nHoliday feature improvement: {improvement:+.2f}%")
print("="*70)

# Save LSTM results for inference notebook
lstm_summary = {
    'algorithm': 'LSTM',
    'avg_rmse_no_holiday': lstm_results['avg_rmse_no_holiday'],
    'avg_rmse_with_holiday': lstm_results['avg_rmse_with_holiday'],
    'avg_mape_no_holiday': avg_mape_no_holiday,
    'avg_mape_with_holiday': avg_mape_with_holiday,
    'markets': market_columns,
    'results': lstm_results
}

joblib.dump(lstm_summary, '../results/metrics/lstm_summary.pkl')
joblib.dump(lstm_results, '../results/metrics/lstm_detailed_results.pkl')

print('\n✓ LSTM results saved to results/metrics/')
print(f'✓ Average RMSE (no holiday): {lstm_summary["avg_rmse_no_holiday"]:.2f}')
print(f'✓ Average RMSE (with holiday): {lstm_summary["avg_rmse_with_holiday"]:.2f}')
print(f'✓ Average MAPE (no holiday): {avg_mape_no_holiday:.2f}%')
print(f'✓ Average MAPE (with holiday): {avg_mape_with_holiday:.2f}%')


LSTM FINAL SUMMARY:
LSTM (no holidays):
  Average RMSE: 11,933.27
  Average MAPE: 13.76%

LSTM (with holidays):
  Average RMSE: 14,497.90
  Average MAPE: 18.02%

Holiday feature improvement: -21.49%

✓ LSTM results saved to results/metrics/
✓ Average RMSE (no holiday): 11933.27
✓ Average RMSE (with holiday): 14497.90
✓ Average MAPE (no holiday): 13.76%
✓ Average MAPE (with holiday): 18.02%
